In [2]:
import cv2
import numpy as np
i=0
key = cv2.waitKey(1)
webcam = cv2.VideoCapture(0)

while True:

    check, frame = webcam.read()
#   for i in range(0, 5):
    frame = cv2.flip(frame,1)
    framecp = frame.copy()
    cv2.rectangle(framecp,(390,80),(640,380),(70,0,70), 3)
    cv2.circle(framecp, (515,230), 3, (70,0,70), 3)
    cv2.imshow("Capturing", framecp)
    subimg = frame[80:380,390:640]
    img_new = cv2.cvtColor(subimg, cv2.COLOR_BGR2GRAY)
    img_n = cv2.cvtColor(subimg, cv2.COLOR_BGR2HSV)
    img_n = cv2.GaussianBlur(img_n,(7,7),cv2.BORDER_DEFAULT)
    hr,wc = img_n.shape
    cv2.imshow("Testing",img_n)
    thres,_,_,_ = cv2.mean(img_new[123:127,98:102])
    h,w = img_new.shape
    rng = 40
    for i in range(h):
        for j in range(w):
            if img_new[i,j]>thres+rng:
                img_new[i,j] = 0
            elif img_new[i,j]<thres-rng:
                img_new[i,j] = 0
            else:
                img_new[i,j] = 255
    cv2.imshow("Sub Image",img_new)
    key = cv2.waitKey(1)
         
    if key == ord('s'):
        cv2.imwrite(filename=str(i)+'_saved_img.jpg', img=subimg)
        webcam.release()
        img_new = cv2.imread(str(i)+'_saved_img.jpg', cv2.IMREAD_GRAYSCALE)
        cv2.imshow("Captured Image", img_new)
       # cv2.waitKey(1650)
        #cv2.destroyAllWindows()
        #print("Processing image...")
        #img_new = cv2.imread(str(i)+'_saved_img.jpg', cv2.IMREAD_ANYCOLOR)
        #print("Converting RGB image to grayscale...")
        #gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
        #print("Converted RGB image to grayscale...")
        #print("Converting Gray image to Binary...")
        #retval, dimage = cv2.threshold(img_new, 120, 255, cv2.THRESH_BINARY)
        #cv2.imshow("Testing",dimage)
        thres,_,_,_ = cv2.mean(img_new[123:127,98:102])
        h,w = img_new.shape
        rng = 40
        for i in range(h):
            for j in range(w):
                if img_new[i,j]>thres+rng:
                    img_new[i,j] = 0
                elif img_new[i,j]<thres-rng:
                    img_new[i,j] = 0
                else:
                    img_new[i,j] = 255
            
        cv2.imshow("Thresholded",img_new)
        kernel = np.ones((2,2), np.uint8)
        img_erode = cv2.erode(img_new, kernel, iterations = 3)
        img_dilate = cv2.dilate(img_erode, kernel, iterations = 3)
        cv2.imshow("dilated",img_dilate)
        #print("Converted Gray image to Binary...")
        #print("Resizing image to 28x28 scale...")
        img_ = cv2.resize(img_new, (224, 224))
        #print("Resized...")
        img_resized = cv2.imwrite(filename=str(i)+'_saved_img-final.jpg', img=img_)
        cv2.imwrite(filename=str(i)+'_saved_img-dialated.jpg', img=img_dilate)
        #print("Image saved!")
        i = i+1
        webcam = cv2.VideoCapture(0)

    elif key == ord('q'):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break


ValueError: too many values to unpack (expected 2)

In [6]:
import cv2
import numpy as np
from PIL import Image, ImageOps
from keras.preprocessing import image
from keras.models import load_model
i=0
key = cv2.waitKey(1)
webcam = cv2.VideoCapture(0)
saved_path = "VGG_cross_validated.h5"
model = load_model(saved_path)
font = cv2.FONT_HERSHEY_SIMPLEX
while True:

    check, frame = webcam.read()
    frame = cv2.flip(frame,1)
    framecp = frame.copy()
    kernel = np.ones((3,3),np.uint8)
    cv2.rectangle(framecp,(390,80),(640,380),(70,0,70), 3)
   # cv2.circle(framecp, (515,230), 3, (70,0,70), 3)
    subimg = frame[80:380,390:640]
    hsv = cv2.cvtColor(subimg, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0,20,70], dtype=np.uint8)
    upper_skin = np.array([20,255,255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    mask = cv2.dilate(mask,kernel,iterations = 4)
    mask = cv2.GaussianBlur(mask,(5,5),100)
    gs = mask
    mask = cv2.threshold(mask, 0, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY)[1]
    thrs = mask
    mask = cv2.erode(mask,kernel,iterations = 3)
    er=mask
    mask = cv2.dilate(mask,kernel,iterations = 3)
    dil=mask
    imgs = Image.fromarray(mask)
    imgs = imgs.convert('RGB')
    imgs = ImageOps.fit(imgs, (224,224), Image.ANTIALIAS)
    imgs = np.expand_dims(imgs,axis=0)
    val = model.predict(imgs)
    res = np.where(val == np.amax(val))
    
    if res[1] == [0]:
        cv2.putText(framecp,"Fist",(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    elif res[1] == [1]:
        cv2.putText(framecp,"L",(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    elif res[1] == [2]:
        cv2.putText(framecp,"OK",(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    elif res[1] == [3]:
        cv2.putText(framecp,"Palm",(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    elif res[1] == [4]:
        cv2.putText(framecp,"Peace",(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    cv2.imshow("Testing",mask)
    cv2.imshow("Capturing", framecp)
    key = cv2.waitKey(1)
    if key == ord('s'):
        cv2.imwrite(filename=str(i)+'_framecp_img.jpg', img=framecp)
        cv2.imwrite(filename=str(i)+'_main_img.jpg', img=subimg)
       # cv2.imwrite(filename=str(i)+'_hsv_img.jpg', img=hsv)
        #cv2.imwrite(filename=str(i)+'_gauss_img.jpg', img=gs)
        #cv2.imwrite(filename=str(i)+'_thrs_img.jpg', img=thrs)
        #cv2.imwrite(filename=str(i)+'_er_img.jpg', img=er)
        cv2.imwrite(filename=str(i)+'_dil_img.jpg', img=dil)
        i=i+1
    if key == ord('q'):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break

ModuleNotFoundError: No module named 'PIL'

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [1]:
tf.math.confusion_matrix()




NameError: name 'tf' is not defined

In [4]:
import tensorflow as tf

tf.math.confusion_matrix(Confusion, model)


NameError: name 'Confusion' is not defined

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image, ImageOps
from keras.preprocessing import image
from keras.models import load_model
i=0
key = cv2.waitKey(1)
webcam = cv2.VideoCapture(0)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(52, activation=tf.nn.softmax)
])
model.compile(optimizer = tf.train.AdamOptimizer(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.load_weights("model.h5")
font = cv2.FONT_HERSHEY_SIMPLEX
while True:

    check, frame = webcam.read()
    frame = cv2.flip(frame,1)
    framecp = frame.copy()
    kernel = np.ones((3,3),np.uint8)
    cv2.rectangle(framecp,(390,80),(640,380),(70,0,70), 3)
    cv2.circle(framecp, (515,230), 3, (70,0,70), 3)
    subimg = frame[80:380,390:640]
    imgs = Image.fromarray(subimg)
    imgs = imgs.convert('RGB')
    imgs = ImageOps.fit(imgs, (150,150), Image.ANTIALIAS)
    cv2.imshow("Given",np.array(imgs))
    imgs = np.expand_dims(imgs,axis=0)
    val = model.predict(imgs)
    res = np.where(val == np.amax(val))
    cv2.putText(framecp,str(res[1]),(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
    cv2.imshow("Capturing", framecp)
    key = cv2.waitKey(1)
    if key == ord('q'):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break